## 

In [1]:
import face_alignment
from skimage import io
import cv2

# gpu버전
# fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False)
# cpu버전
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, device='cpu')

In [3]:
# file_dir = 'sample.png'
# file_dir='./dataset/data_fullface/0003/003_12_image.png'
file_dir='./test/assets/f_50_fron.jpg'

input = io.imread(file_dir)
preds = fa.get_landmarks(input)

In [4]:
# 사진 띄우기

cv2.imshow('image', input)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
# 랜드마크 하나씩 출력해서 확인하기

n=0
print(preds[n][8][0], preds[n][8][1])  # 턱끝
print(preds[n][28][0], preds[n][28][1])  # 위쪽 코
print(preds[n][0][0], preds[n][0][1])  # 왼쪽 광대(끝)
print(preds[n][16][0], preds[n][16][1])  # 오른쪽 광대(끝)

144.0 291.0
141.0 193.0
58.0 184.0
224.0 193.0


In [7]:
# 깨끗한 이미지
# file_dir='./dataset/data_fullface/0003/003_00_image.png'
# file_dir='./dataset/data_fullface/0003/003_12_image.png'
# file_dir = './dataset/data_fullface/0012/012_11_image.png'
file_dir='./test/assets/f_50_fron.jpg'
input = io.imread(file_dir)
preds = fa.get_landmarks(input)


half_height=preds[n][8][1] - preds[n][28][1]
# img = cv2.line(input, (x_min, y_min), (x_max, y_max), (0, 255, 0), 3) # (B, G, R)
# img = cv2.line(input, (preds[n][0][0], preds[n][0][1]), (preds[n][8][0], preds[n][8][1]), (0, 0, 255), 3) # (B, G, R)
img = cv2.line(input, (preds[n][0][0], preds[n][8][1]), (preds[n][16][0], preds[n][8][1]), (0, 0, 255), 3)
img = cv2.line(input, (preds[n][16][0], preds[n][8][1]), (preds[n][16][0], preds[n][28][1]-half_height), (0, 0, 255), 3)
img = cv2.line(input, (preds[n][16][0], preds[n][28][1]-half_height), (preds[n][0][0], preds[n][28][1]-half_height), (0, 0, 255), 3)
img = cv2.line(input, (preds[n][0][0], preds[n][28][1]-half_height), (preds[n][0][0], preds[n][8][1]), (0, 0, 255), 3)


cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
# crop roi image

img = cv2.imread(file_dir)

roi_image = img.copy() 
roi_image = img[int(preds[n][28][1]-half_height):int(preds[n][8][1]+1), int(preds[n][0][0]):int(preds[n][16][0]+1)]

cv2.imshow("src", img)
cv2.imshow("dst", roi_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
# resize

resized = cv2.resize(roi_image, dsize=(120, 120), interpolation=cv2.INTER_AREA) # x, y

print(img.shape)
print(roi_image.shape)
print(resized.shape)

# cv2.imshow("src", roi_image)
cv2.imshow("dst", resized)

cv2.waitKey(0)
cv2.destroyAllWindows()

(480, 270, 3)
(197, 167, 3)
(120, 120, 3)


In [ ]:
# save
print(file_dir)
print(file_dir[:-4])

save_dir = file_dir[:-4] + '_resized.jpg'
print(save_dir)

In [ ]:
cv2.imwrite(save_dir, resized)

## 얼굴 영역만 검출한 후 crop해서 저장하기

In [ ]:
import os
import glob
import cv2
import numpy as np
import face_alignment
from skimage import io

fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False)
n = 0
PRLAB_path='./dataset/data_fullface' 
PRLAB_folder_list= os.listdir(PRLAB_path) #PRLAB 내부 폴더 리스트
#print (PRLAB_folder_list)

for folder_num in range(0, len(PRLAB_folder_list)):
    #print('PRLAB_folder: %s' %PRLAB_folder_list[folder_num])
    dirs = PRLAB_path + '/' + PRLAB_folder_list[folder_num]
#     print('dirs: %s' %dirs)
    filelist = os.listdir(dirs)
    
    for img_num in range(0,len(filelist)):
        file_path = dirs + '/' + filelist[img_num]
        stream = open(file_path.encode("utf-8"), "rb")
        bytes = bytearray(stream.read())
        numpyArray = np.asarray(bytes, dtype=np.uint8)
        try:
            img = cv2.imdecode(numpyArray,cv2.IMREAD_COLOR) # 이미지가 아니라 다른게 불러와지면 에러 발생
            preds = np.array(fa.get_landmarks(img)) # 검출 불가능하면 에러 발생
        except:
            print("face detect불가")
            print(dirs +'/'+ filelist[img_num])
            print("폴더명 :"+ str(folder_num))
            continue
        
#         print('filelist: %s' %filelist)
        

        x_max = 0
        x_min = len(img)
        y_max = 0
        y_min = len(img[0])
        
        try:
            for i in range(0, len(preds[0])):
                if (x_max < preds[0][i][0]):
                    x_max = preds[0][i][0]
                if (x_min > preds[0][i][0]):
                    x_min = preds[0][i][0]
                if (y_max < preds[0][i][1]):
                    y_max = preds[0][i][1]
                if (y_min > preds[0][i][1]):
                    y_min = preds[0][i][1]
        except: 
            print("periocular detect불가")
            print(dirs +'/'+ filelist[img_num])
            print("폴더명 :"+ str(folder_num))
            continue

        half_height=preds[n][8][1] - preds[n][28][1]    
        roi_image = img.copy() 
        roi_image = img[int(preds[n][28][1]-half_height):int(preds[n][8][1]+1), int(preds[n][0][0]):int(preds[n][16][0]+1)]
        try:
            resized = cv2.resize(roi_image, dsize=(120, 120), interpolation=cv2.INTER_AREA)
        except: 
            print("face detect불가")
            print(dirs +'/'+ filelist[img_num])
            print("폴더명 :"+ str(folder_num))
            
            continue
        
        if (os.path.isdir('./output')): # output폴더 있으면
            out_folder = './output/'+ str(folder_num)+'/'
            
        if(os.path.isdir(out_folder)): #output/person 에 save하기
            save_fname = str(folder_num) + '_' + str(img_num) + '_resized.jpg'
            cv2.imwrite('./output/'+ str(folder_num) + '/' + save_fname, resized)
            #print('./output/'+ str(folder_num) + '/' + save_fname)
        else: # output/person 이 없으면 만들고 save
            os.mkdir(out_folder)
            save_fname = str(folder_num) + '_' + str(img_num) + '_resized.jpg'
            cv2.imwrite('./output/'+ str(folder_num) + '/' + save_fname, resized)
            #print('./output/'+ str(folder_num) + '/' + save_fname)
